In [2]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import os
from uuid import uuid1


In [3]:
# Initialize and display the camera
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=300, height=300)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [6]:
# create and display buttons that we will use to save snapshots for each class level
# also display text boxes demonstrating how many images of each category that we have collected so far
# we will target at least 50 balloon images + 50 no balloon images

button_layout = widgets.Layout(width='128px', height='64px')
balloon_button = widgets.Button(description='add balloon', button_style='success', layout=button_layout)
no_balloon_button = widgets.Button(description='add no balloon', button_style='danger', layout=button_layout)
balloon_count = widgets.IntText(layout=button_layout, value=len(os.listdir(balloon_dir)))
no_balloon_count = widgets.IntText(layout=button_layout, value=len(os.listdir(no_balloon_dir)))

display(widgets.HBox([balloon_count, balloon_button]))
display(widgets.HBox([no_balloon_count, no_balloon_button]))

In [5]:
# setup directories for the balloon & no balloon photos

balloon_dir = 'balloon_dataset/balloon'
no_balloon_dir = 'balloon_dataset/no_balloon'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(balloon_dir)
    os.makedirs(no_balloon_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


In [7]:
# Attach functions to buttons to save images for each category to the button's on-click event
# We save the value fo the Image widget rather than the camera because it's already in compressed jpeg format

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_balloon():
    global balloon_dir, balloon_count
    save_snapshot(balloon_dir)
    balloon_count.value = len(os.listdir(balloon_dir))
    
def save_no_balloon():
    global no_balloon_dir, no_balloon_count
    save_snapshot(no_balloon_dir)
    no_balloon_count.value = len(os.listdir(no_balloon_dir))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
balloon_button.on_click(lambda x: save_balloon())
no_balloon_button.on_click(lambda x: save_no_balloon())

Great! Now the buttons above should save images to the ``free`` and ``blocked`` directories.  You can use the Jupyter Lab file browser to view these files!

Now go ahead and collect some data 

1. Place the robot in a scenario where it's blocked and press ``add blocked``
2. Place the robot in a scenario where it's free and press ``add free``
3. Repeat 1, 2

> REMINDER: You can move the widgets to new windows by right clicking the cell and clicking ``Create New View for Output``.  Or, you can just re-display them
> together as we will below

Here are some tips for labeling data

1. Try different orientations
2. Try different lighting
3. Try varied object / collision types; walls, ledges, objects
4. Try different textured floors / objects;  patterned, smooth, glass, etc.

Ultimately, the more data we have of scenarios the robot will encounter in the real world, the better our collision avoidance behavior will be.  It's important
to get *varied* data (as described by the above tips) and not just a lot of data, but you'll probably need at least 100 images of each class (that's not a science, just a helpful tip here).  But don't worry, it goes pretty fast once you get going :)

In [8]:
display(image)
display(widgets.HBox([balloon_count, balloon_button]))
display(widgets.HBox([no_balloon_count, no_balloon_button]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## Next

Once you've collected enough data, we'll need to copy that data to our GPU desktop or cloud machine for training.  First, we can call the following *terminal* command to compress
our dataset folder into a single *zip* file.

> The ! prefix indicates that we want to run the cell as a *shell* (or *terminal*) command.

> The -r flag in the zip command below indicates *recursive* so that we include all nested files, the -q flag indicates *quiet* so that the zip command doesn't print any output

In [10]:
!zip -r -q balloon_dataset.zip balloon_dataset

You should see a file named ``dataset.zip`` in the Jupyter Lab file browser.  You should download the zip file using the Jupyter Lab file browser by right clicking and selecting ``Download``.

Next, we'll need to upload this data to our GPU desktop or cloud machine (we refer to this as the *host*) to train the collision avoidance neural network.  We'll assume that you've set up your training
machine as described in the JetBot WiKi.  If you have, you can navigate to ``http://<host_ip_address>:8888`` to open up the Jupyter Lab environment running on the host.  The notebook you'll need to open there is called ``collision_avoidance/train_model.ipynb``.

So head on over to your training machine and follow the instructions there!  Once your model is trained, we'll return to the robot Jupyter Lab enivornment to use the model for a live demo!